In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq

In [2]:
#get table names for CCG measures where staging table = live measure table

sql = """
SELECT
 staging.*
FROM
 staging_measures.INFORMATION_SCHEMA.TABLES as staging --staging table schema
 join measures.INFORMATION_SCHEMA.TABLES as live --live table schema
on staging.table_name=live.table_name --join tables
 where
 staging.table_name like 'ccg%'
 order by staging.creation_time DESC

"""
table_df = bq.cached_read(sql, csv_path='staging_df.csv', use_cache=False)

C:\Users\racro\Anaconda3\lib\site-packages\pandas\io\gbq.py:108: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  **kwargs)


In [3]:
##create dataframe consisting of all staging and live measure, calculating difference between two
staging_df=pd.DataFrame()

#create for next loop to go through each table name in the previous query
for table_name in table_df['table_name']:
    
    sql = """
    SELECT
      '{}' AS table_name, --selects current table name in for next loop
      ROUND(SUM(a.numerator)-SUM(b.numerator),0) AS num_dif, --calculates difference between staging and live numerators
      ROUND(SUM(a.denominator)-SUM(b.denominator),0) AS den_dif --calculates difference between staging and live denominators
    FROM
      `ebmdatalab.staging_measures.{}` AS a
    INNER JOIN
      `ebmdatalab.measures.{}` AS b
    ON
      a.month = b.month
      AND a.pct_id = b.pct_id
    GROUP BY
      table_name
    """
    
    sql = sql.format(table_name, table_name, table_name) #using python string to add table_name to SQL
    #concatenate each table name into single file during for next loop
    staging_df = pd.concat([staging_df, bq.cached_read(sql, '{}_staging_df.csv'.format(table_name), use_cache=True)])
    #save as csv
    staging_df.to_csv("stagingtest.csv")

In [5]:
#removes any rows where the staging table is the same as live table, showing just where there are differences
staging_df.loc[(staging_df["num_dif"]!=0) | (staging_df["den_dif"]!=0)]

,table_name,num_dif,den_dif
0,ccg_data_ppi,0.0,328290.0
0,ccg_data_lpglucosamine,-283.0,0.0
0,ccg_data_lpbathshoweremollients,30257952.0,0.0
